In [ ]:
# API token and CA crt

$serviceAccount = "cp-api-explorer"
# get right secret
$secretName = kubectl get serviceaccount cp-api-explorer -o json | ConvertFrom-Json `
    | % { $_.secrets } `
    | ? { $_.name.startsWith("$($serviceAccount)-token-") } `
    | select -First 1 | % { $_.name }
$secretName
# secret from secretName
$token = kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data.token | base64 -d }
# ca.crt
$caCrtFilename = "./k8sca.crt"
kubectl get secret $secretName -o json | ConvertFrom-Json | % { $_.data."ca.crt" | base64 -d } | Out-File $caCrtFilename
$caCrtFilename 

$apiServerUrl = kubectl config view -o json | jq -c '.clusters[] | select(.name == ""minikube"") | .cluster.server'
$apiServerUrl

cp-api-explorer-token-zg7g7
./k8sca.crt
"https://127.0.0.1:51433"


In [ ]:
# test - name of first pod in default NS
/usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/default/pods" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[0] | .metadata.name'

web-1


In [ ]:
# list of namespaces
$namespaces = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces" `
   --header "Authorization: Bearer $token" --cacert $caCrtFilename `
   | jq -r '.items[] | .metadata.name'
$namespaces

default
kube-node-lease
kube-public
kube-system


In [ ]:
# all pods per namespace with IP address
$namespaces | % {
    $namespace = $_
    /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
}


namespace   name                    ip
---------   ----                    --
default     web-1                   172.17.0.5
default     web-10                  172.17.0.13
default     web-2                   172.17.0.4
default     web-3                   172.17.0.7
default     web-4                   172.17.0.6
default     web-5                   172.17.0.10
default     web-6                   172.17.0.8
default     web-7                   172.17.0.9
default     web-8                   172.17.0.11
default     web-9                   172.17.0.12
default     web01                   172.17.0.3
kube-system coredns-64897985d-wc4c9 172.17.0.2
kube-system etcd-minikube           192.168.49…
kube-system kube-apiserver-minikube 192.168.49…
kube-system kube-controller-manage… 192.168.49…
kube-system kube-proxy-t59s8        192.168.49…
kube-system kube-scheduler-minikube 192.168.49…
kube-system storage-provisioner     192.168.49…



In [ ]:
# all pods per namespace with IP address
$objects = $namespaces | % {
    $namespace = $_
    $pods = /usr/bin/curl -s "$apiServerUrl/api/v1/namespaces/$namespace/pods" `
        --header "Authorization: Bearer $token" --cacert $caCrtFilename `
        | ConvertFrom-Json | % { $_.items } | % {
            [PSCustomObject] @{
                namespace = $namespace;
                name = $_.metadata.name;
                ip = $_.status.podIPs[0].ip
            }
        }
    $ips = $pods | % { $_.ip }
    if ($ips.Length -gt 0) { 
        # Write-Host $namespace ": " $ips 
        [PSCustomObject] @{
            name = "ns-$namespace-ips";
            id = "id-ns-$namespace-ips";
            description = "IP adresses in namespace $namespace";
            ranges = $ips
        }
    }
}

$objects


name               id                    description                          ranges
----               --                    -----------                          ------
ns-default-ips     id-ns-default-ips     IP adresses in namespace default     {172.17.0.5, 172.17.…
ns-kube-system-ips id-ns-kube-system-ips IP adresses in namespace kube-system {172.17.0.2, 192.168…



In [ ]:
[PSCustomObject] @{
        "version" = "1.0";
        "description" = "Generic Data Center file example";
        "objects" = $objects
} | ConvertTo-Json -Depth 10 | jq .

{
  "version": "1.0",
  "description": "Generic Data Center file example",
  "objects": [
    {
      "name": "ns-default-ips",
      "id": "id-ns-default-ips",
      "description": "IP adresses in namespace default",
      "ranges": [
        "172.17.0.5",
        "172.17.0.13",
        "172.17.0.4",
        "172.17.0.7",
        "172.17.0.6",
        "172.17.0.10",
        "172.17.0.8",
        "172.17.0.9",
        "172.17.0.11",
        "172.17.0.12",
        "172.17.0.3"
      ]
    },
    {
      "name": "ns-kube-system-ips",
      "id": "id-ns-kube-system-ips",
      "description": "IP adresses in namespace kube-system",
      "ranges": [
        "172.17.0.2",
        "192.168.49.2",
        "192.168.49.2",
        "192.168.49.2",
        "192.168.49.2",
        "192.168.49.2",
        "192.168.49.2"
      ]
    }
  ]
}
